In [2]:
# !pip install keras keras-preprocessing pandas scipy -q
!pip install tensorflow -q

# PlantVillage Dataset

In [4]:
!git clone https://github.com/spMohanty/PlantVillage-Dataset plantvillage_dataset

TRAIN_PATH = 'plantvillage_dataset/raw/segmented'
VAL_PATH = 'plantvillage_dataset/raw/segmented_val'

Cloning into 'dataset'...
remote: Enumerating objects: 163235, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 163235 (delta 2), reused 0 (delta 0), pack-reused 163229 (from 1)
Receiving objects: 100% (163235/163235), 2.00 GiB | 30.39 MiB/s, done.
Resolving deltas: 100% (101/101), done.
Updating files: 100% (182401/182401), done.


# Sugarcane Dataset

In [1]:
# upload kaggle.json first.
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [2]:
!apt update -qq
!apt install -qq unzip
!kaggle datasets download nirmalsankalana/sugarcane-leaf-disease-dataset
!unzip -q sugarcane-leaf-disease-dataset.zip -d sugarcane_dataset_train

TRAIN_PATH = 'sugarcane_dataset_train'
VAL_PATH = 'sugarcane_dataset_test'

# Code to split into train and test

In [ ]:
import os
import shutil
import random
from pathlib import Path

# Paths
TRAIN_DIR = Path(TRAIN_PATH)
VAL_DIR = Path(VAL_PATH)
SPLIT_RATIO = 0.2  # 20% goes to val

# Create val/ directory structure
for class_dir in TRAIN_DIR.iterdir():
    if class_dir.is_dir():
        val_class_dir = VAL_DIR / class_dir.name
        val_class_dir.mkdir(parents=True, exist_ok=True)

        # Get all images in the class folder
        images = list(class_dir.glob("*.*"))
        random.shuffle(images)

        val_count = int(len(images) * SPLIT_RATIO)
        val_images = images[:val_count]

        # Move images to val/class_name/
        for img in val_images:
            shutil.move(str(img), str(val_class_dir / img.name))

print("✅ Split complete. 20% moved to val/")


# TRain model

In [1]:
from model import load_model, train_model

In [2]:
num_classes = 5
model = load_model(num_classes)

In [3]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 224, 224, 3)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1_conv1 (Conv2D)     │ (None, 111, 111, 32)   │            864 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1_conv1_bn           │ (None, 111, 111, 32)   │            128 │ block1_conv1[0][0]     │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1_conv1_act          │ (None, 111, 111, 32)   │              0 │ block1_conv1_bn[0][0]  │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1_conv2 (Conv2D)     │ (None, 109, 109, 64)   │         18,432 │ block1_conv1_act[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1_conv2_bn           │ (None, 109, 109, 64)   │            256 │ block1_conv2[0][0]     │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1_conv2_act          │ (None, 109, 109, 64)   │              0 │ block1_conv2_bn[0][0]  │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block2_sepconv1           │ (None, 109, 109, 128)  │          8,768 │ block1_conv2_act[0][0] │
│ (SeparableConv2D)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block2_sepconv1_bn        │ (None, 109, 109, 128)  │            512 │ block2_sepconv1[0][0]  │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block2_sepconv2_act       │ (None, 109, 109, 128)  │              0 │ block2_sepconv1_bn[0]… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block2_sepconv2           │ (None, 109, 109, 128)  │         17,536 │ block2_sepconv2_act[0… │
│ (SeparableConv2D)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block2_sepconv2_bn        │ (None, 109, 109, 128)  │            512 │ block2_sepconv2[0][0]  │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d (Conv2D)           │ (None, 55, 55, 128)    │          8,192 │ block1_conv2_act[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block2_pool               │ (None, 55, 55, 128)    │              0 │ block2_sepconv2_bn[0]… │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization  

 Total params: 153,393,323 (585.15 MB)

 Trainable params: 153,234,011 (584.54 MB)

 Non-trainable params: 159,312 (622.31 KB)

In [5]:
import keras
keras.config.disable_traceback_filtering()

model, history = train_model(model, TRAIN_PATH, VAL_PATH)

Found 2019 images belonging to 5 classes.
Found 502 images belonging to 5 classes.
Epoch 1/35
7/7 ━━━━━━━━━━━━━━━━━━━━ 121s 2s/step - loss: 1.7206 - out1_accuracy: 0.1971 - out1_loss: 1.7817 - out2_accuracy: 0.2381 - out2_loss: 1.6799 - val_loss: 1.6201 - val_out1_accuracy: 0.1562 - val_out1_loss: 1.6399 - val_out2_accuracy: 0.2188 - val_out2_loss: 1.6069
Epoch 2/35
7/7 ━━━━━━━━━━━━━━━━━━━━ 7s 946ms/step - loss: 1.6604 - out1_accuracy: 0.2041 - out1_loss: 1.6566 - out2_accuracy: 0.1880 - out2_loss: 1.6630 - val_loss: 1.6094 - val_out1_accuracy: 0.2188 - val_out1_loss: 1.6184 - val_out2_accuracy: 0.2188 - val_out2_loss: 1.6034
Epoch 3/35
7/7 ━━━━━━━━━━━━━━━━━━━━ 7s 962ms/step - loss: 1.6991 - out1_accuracy: 0.1915 - out1_loss: 1.6861 - out2_accuracy: 0.1389 - out2_loss: 1.7079 - val_loss: 1.6132 - val_out1_accuracy: 0.1562 - val_out1_loss: 1.6270 - val_out2_accuracy: 0.2188 - val_out2_loss: 1.6040
Epoch 4/35
7/7 ━━━━━━━━━━━━━━━━━━━━ 7s 958ms/step - loss: 1.5677 - out1_accuracy: 0.2981 -

Check if the model has been saved!
